<a href="https://colab.research.google.com/github/Phionanamugga/Autonomous_Navigation/blob/main/ObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_drive_0047/2011_10_03_drive_0047_sync.zip


--2025-08-11 15:18:47--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_drive_0047/2011_10_03_drive_0047_sync.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.135.51, 3.5.136.90, 52.219.140.31, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.135.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3103291675 (2.9G) [application/zip]
Saving to: ‘2011_10_03_drive_0047_sync.zip’

2011_10_03_drive_00 100%[===================>]   2.89G  23.8MB/s    in 2m 5s   

2025-08-11 15:20:53 (23.6 MB/s) - ‘2011_10_03_drive_0047_sync.zip’ saved [3103291675/3103291675]



In [2]:
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_calib.zip

--2025-08-11 15:21:06--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_10_03_calib.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.47.231, 52.219.169.113, 3.5.134.58, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.47.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4075 (4.0K) [application/zip]
Saving to: ‘2011_10_03_calib.zip’

2011_10_03_calib.zi 100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-08-11 15:21:07 (285 MB/s) - ‘2011_10_03_calib.zip’ saved [4075/4075]



In [3]:
!unzip -q '*.zip'


2 archives were successfully processed.


In [33]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import torch
import cv2

In [6]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)

In [7]:
!wget https://github.com/itberrios/CV_tracking/raw/main/kitti_tracker/kitti_utils.py

--2025-08-11 15:22:29--  https://github.com/itberrios/CV_tracking/raw/main/kitti_tracker/kitti_utils.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/itberrios/CV_tracking/main/kitti_tracker/kitti_utils.py [following]
--2025-08-11 15:22:29--  https://raw.githubusercontent.com/itberrios/CV_tracking/main/kitti_tracker/kitti_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9759 (9.5K) [text/plain]
Saving to: ‘kitti_utils.py’

kitti_utils.py      100%[===================>]   9.53K  --.-KB/s    in 0s      

2025-08-11 15:22:29 (135 MB/s) - ‘kitti_utils.py’ saved [9759/9759]



In [8]:
DATA_PATH = r'2011_10_03/2011_10_03_drive_0047_sync'
# get RGB camera data
left_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_02/data/*png')))
right_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_03/data/*png')))

# getting LiDAR data
bin_paths = sorted(glob(os.path.join(DATA_PATH, 'velodyne_points/data/*bin')))

# Get GPS/IMU data
oxts_paths = sorted(glob(os.path.join(DATA_PATH, r'oxts/data/*txt')))

print(f'Number of RGB images: {len(left_image_paths)}')
print(f'Number of RGB images: {len(right_image_paths)}')
print(f'Number of LiDAR points: {len(bin_paths)}')
print(f'Number of GPS/IMU data: {len(oxts_paths)}')

Number of RGB images: 837
Number of RGB images: 837
Number of LiDAR points: 837
Number of GPS/IMU data: 837


In [10]:
# Loading camera calibration data
with open('2011_10_03/calib_cam_to_cam.txt', 'r') as f:
    calib = f.readlines()

# Get projection matrices (rectified left camera ---> left camera (u,v,z))
p_rect_cam2 = np.array([float(x) for x in calib[25].strip().split()[1:]]).reshape(3, 4)

# Get rectified rotation matrices (left camera ---> rectified left camera)
R_ref_rect2 = np.array([float(x) for x in calib[24].strip().split()[1:]]).reshape(3, 3)

# Add (0,0,0) translation and convert to homogeneous coordinates
R_ref_rect2 = np.insert(R_ref_rect2, 3, values=[0,0,0], axis=1)
R_ref_rect2 = np.insert(R_ref_rect2, 3, values=[0,0,0,1], axis=0)

# Get rigid transformation from camera 0 (ref) to camera 2
R_2 = np.array([float(x) for x in calib[21].strip().split()[1:]]).reshape(3, 3)
t_2 = np.array([float(x) for x in calib[22].strip().split()[1:]]).reshape(3, 1)

# Get cam0 to cam2 rigid body transformation in homogeneous coordinates
T_Cam0_cam2 = np.insert(np.hstack((R_2, t_2)), 3, values=[0,0,0,1], axis=0)

In [16]:
def get_rid_transformation(calib_path):
    with open(calib_path, 'r') as f:
        calib = f.readlines()
    R = np.array([float(x) for x in calib[1].strip().split()[1:]]).reshape(3, 3)
    t = np.array([float(x) for x in calib[2].strip().split()[1:]])[:,None]

    T = np.vstack((np.hstack((R, t)), [0, 0, 0, 1]))
    return T
    T_velo_ref = get_rid_tranformation(r'2011_10_03/calib_velo_to_cam.txt')
    T_velo_imu = get_rid_tranformation(r'2011_10_03/calib_velo_to_imu.txt')

    print("T_velo_ref (LiDAR to Camera):\n", T_velo_ref)
    print("T_velo_imu (LiDAR to IMU):\n", T_velo_imu)


In [18]:
T_velo_ref

array([[ 7.967514e-03, -9.999679e-01, -8.462264e-04, -1.377769e-02],
       [-2.771053e-03,  8.241710e-04, -9.999958e-01, -5.542117e-02],
       [ 9.999644e-01,  7.969825e-03, -2.764397e-03, -2.918589e-01],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

In [23]:

# Function to load rigid body transformation from calibration file
def get_rid_transformation(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Calibration file not found: {file_path}")
    with open(file_path, 'r') as f:
        calib = f.readlines()
    R, t = None, None
    for line in calib:
        if line.startswith('R:'):
            R = np.array([float(x) for x in line.strip().split()[1:]]).reshape(3, 3)
        elif line.startswith('T:') or line.startswith('t:'):
            t = np.array([float(x) for x in line.strip().split()[1:]]).reshape(3, 1)
    if R is None or t is None:
        raise ValueError(f"Invalid format in {file_path}: 'R:' or 'T:'/'t:' not found")
    T = np.vstack((np.hstack((R, t)), [0, 0, 0, 1]))
    return T

# Loading camera calibration data
calib_file = '2011_10_03/calib_cam_to_cam.txt'
if not os.path.exists(calib_file):
    raise FileNotFoundError(f"Calibration file not found: {calib_file}")
with open(calib_file, 'r') as f:
    calib = f.readlines()

# Get projection matrices (rectified left camera ---> left camera (u,v,z))
p_rect_cam2 = np.array([float(x) for x in calib[25].strip().split()[1:]]).reshape(3, 4)

# Get rectified rotation matrices (left camera ---> rectified left camera)
R_ref_rect2 = np.array([float(x) for x in calib[24].strip().split()[1:]]).reshape(3, 3)

# Add (0,0,0) translation and convert to homogeneous coordinates
R_ref_rect2 = np.insert(R_ref_rect2, 3, values=[0,0,0], axis=1)
R_ref_rect2 = np.insert(R_ref_rect2, 3, values=[0,0,0,1], axis=0)

# Get rigid transformation from camera 0 (ref) to camera 2
R_2 = np.array([float(x) for x in calib[21].strip().split()[1:]]).reshape(3, 3)
t_2 = np.array([float(x) for x in calib[22].strip().split()[1:]]).reshape(3, 1)

# Get cam0 to cam2 rigid body transformation in homogeneous coordinates
T_Cam0_cam2 = np.vstack((np.hstack((R_2, t_2)), [0,0,0,1]))

# Loading LiDAR calibration data
T_velo_ref = get_rid_transformation('2011_10_03/calib_velo_to_cam.txt')

# Skip loading T_velo_imu since the file is missing and not used in transformations
# If needed, uncomment and ensure the file exists:
# T_velo_imu = get_rid_transformation('2011_10_03/calib_velo_to_imu.txt')

# Transform from Velodyne LiDAR to left color camera (shape 3x4)
T_velo_cam2 = p_rect_cam2 @ R_ref_rect2 @ T_Cam0_cam2 @ T_velo_ref

# Homogeneous transform from left color camera to Velodyne LiDAR (shape 4x4)
T_cam2_velo = np.linalg.inv(np.vstack((T_velo_cam2, [0,0,0,1])))

# Print results
print("T_velo_cam2 (LiDAR to left color camera, 3x4):\n", T_velo_cam2)
print("T_cam2_velo (left color camera to LiDAR, 4x4):\n", T_cam2_velo)

T_velo_cam2 (LiDAR to left color camera, 3x4):
 [[ 6.07484390e+02 -7.18537361e+02 -1.01875822e+01 -9.55729189e+01]
 [ 1.80027463e+02  5.89922104e+00 -7.20148711e+02 -9.34570810e+01]
 [ 9.99973895e-01  4.85949260e-04 -7.20693419e-03 -2.84637351e-01]]
T_cam2_velo (left color camera to LiDAR, 4x4):
 [[ 5.94945492e-07 -1.00306966e-05  1.00147052e+00  2.84175339e-01]
 [-1.39105363e-03  1.12417451e-05  8.43041553e-01  1.08064679e-01]
 [-1.12463249e-05 -1.39101753e-03  2.57260040e-01 -5.78494660e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [24]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17521, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17521 (delta 9), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17521/17521), 16.62 MiB | 33.18 MiB/s, done.
Resolving deltas: 100% (12005/12005), done.


In [ ]:
!pip install -r yolov5/requirements.txt

In [26]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-8-11 Python-3.11.13 torch-2.6.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 118MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [27]:
# set confidence and IOU thresholds
model.conf = 0.25 # confidence threshold(0.1) ,default:0.25
model.iou = 0.25 # NMS IOU threshold for NMS(0.45)

In [30]:
detections = model(left_image_paths[0])

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [34]:

def get_uvz_centers(image, velo_uvz, bboxes, draw=True):
    # Unpack LiDAR camera coordinates
    u, v, z = velo_uvz[:, 0], velo_uvz[:, 1], velo_uvz[:, 2]

    # Initialize output array: bboxes + 3 columns for (u, v, z)
    bboxes_out = np.zeros((bboxes.shape[0], bboxes.shape[1] + 3))
    bboxes_out[:, :bboxes.shape[1]] = bboxes  # Copy original bboxes

    # Iterate through all detected bounding boxes
    for i, bbox in enumerate(bboxes):
        # Convert bbox coordinates to integers
        pt1 = torch.round(bbox[0:2]).to(torch.int).numpy()  # (y_min, x_min)
        pt2 = torch.round(bbox[2:4]).to(torch.int).numpy()  # (y_max, x_max)

        # Get center location of the object on the image
        obj_x_center = (pt1[1] + pt2[1]) / 2  # x center
        obj_y_center = (pt1[0] + pt2[0]) / 2  # y center

        # Compute distance from LiDAR points to the center
        center_delta = np.abs(np.vstack((u, v)).T - np.array([obj_x_center, obj_y_center]))

        # Choose coordinate pair with the smallest L2 norm
        min_loc = np.argmin(np.linalg.norm(center_delta, axis=1))

        # Get LiDAR location in image/camera space
        velo_depth = z[min_loc]
        uvz_location = np.array([u[min_loc], v[min_loc], velo_depth])

        # Add velo projections (u, v, z) to bboxes_out
        bboxes_out[i, -3:] = uvz_location

        # Draw depth on image at center of each bounding box
        if draw:
            object_center = (int(np.round(obj_x_center)), int(np.round(obj_y_center)))
            cv2.putText(
                image,
                '{:.2f} m'.format(velo_depth),
                object_center,  # Center of bounding box
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),  # Blue color in BGR
                2,
                cv2.LINE_AA
            )

    return bboxes_out

In [36]:
def get_detection_coordinates(image, bin_path, draw_boxes=True, draw_depth =True):
    # compute detections in the left image
    detections = model(left_image_paths[0])

    # draw boxes on images
    if draw_boxes:
        detections.show()


    # get bounding box locations
    bboxes = detections.xyxy[0].cpu().numpy()

    velo_uvz = project_velobin2uvz(bin_path,
                                   T_velo_cam2,
                                   left_image,
                                   remove_plane = True)

    # get uvz centers for detected objects
    bboxes_out = get_uvz_centers(image, velo_uvz, bboxes, draw=draw_depth)

    return bboxes, velo_uvz

In [37]:
!pip install pymap3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.0 MB/s eta 0:00:00


In [39]:
import pymap3d as pm
def imu2geodetic(x,y,z, lat0, lon0, alt0, heading0):

  # convert to RAE
  rng = np.sqrt(x**2 + y**2 + z**2)
  az = np.degrees(np.arctan2(y,x)) + np.degrees(heading0)
  el = np.degrees(np.arctan2(np.sqrt(x**2 + y**2), z)) + 90

  # convert to geodetic
  lla = pm.geodetic2ned(az, el, rng,lat0, lon0, alt0)

  # convert to numpy array
  lla = np.vstack((lla[0], lla[1], lla[2])).T
  return lla


In [ ]:
# Testing the pipeline
index = 10
left_image = cv2.cvtColor(cv2.imread(left_image_paths[index]),cv2.COLOR_BGR2RGB)
bin_path = bin_paths[index]
oxts_frame = get_oxts(oxts_paths[index])

# get detections and object centers in uvz
bboxes, velo_uvz = get_detection_coordinates(left_image, bin_path, draw_boxes=True, draw_depth=True)

# get transformed coordinates of object centers
uvz = bboxes[:, -3:]

# transform to (u,v,z)
imu_uvz = transform_uvz(uvz, T_cam2_imu)

# get Lat/Long on each detected objects
lat0 = oxts_frame[0]
lon0 = oxts_frame[1]
alt0 = oxts_frame[2]
heading0 = oxts_frame[5]



In [ ]:
import mmcv
print(mmcv.__version__)  # Should print 2.1.0
import mmdet
import mmdet3d
print(mmdet.__version__, mmdet3d.__version__)  # Should print 3.3.0, 1.4.0

In [ ]:
from mmdet3d.datasets import build_dataset
cfg.data.train.data_root = './data/kitti/'
dataset = build_dataset(cfg.data.train)

In [ ]:

model = torch.hub.load('pytorch/vision:v0.15.2', 'deeplabv3_resnet101', pretrained=True)
model.eval()


In [ ]:
import torchvision.transforms as T
from PIL import Image
import numpy as np

transform = T.Compose([T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406],
                                               std=[0.229, 0.224, 0.225])])

def segment_image(image_path):
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    sem_prob = torch.softmax(output, dim=0).cpu().numpy()  # shape: [21, H, W]
    return sem_prob


In [ ]:
import numpy as np

def project_lidar_to_image(points, calib):
    # points: Nx3 array (x,y,z)
    # calib: calibration matrix from KITTI
    # Return: Nx2 array of pixel coordinates
    pts_3d = np.hstack((points, np.ones((points.shape[0],1))))
    pts_2d = calib @ pts_3d.T
    pts_2d[:2, :] /= pts_2d[2, :]
    return pts_2d[:2, :].T  # Nx2 pixel coordinates